 # Analysis 20190419
 * Written by: Chien-Cheng Shih
 * Running under conda env "lipidraft"
 * Source Images: testdata

In [0]:
# import resource
import subprocess
import os, sys
from core.fileop import DirCheck, ListFiles, GetPendingList, GetGrpFLs

#Functions ---------------------------------------------------------------
def dict2arg_fiji(arg):
    arg_str = str()
    for key, val in arg.items():
        str_tmp = str(key + '="' + val + '", ')
        arg_str = arg_str + str_tmp
    return arg_str

def list2arg_python(arg):
    arg_str = str()
    for var in arg:
        arg_str = arg_str +  "'" + str(var) + "' " 
    return arg_str
# -------------------------------------------------------------------------


 ## Specify Parameters
 * `fiji`: path to the fiji application
 * `codepath`: the workspace for code/script
 * `datapath`: the workspace for STORM images and data
 * `analysis_dir`: the folder hosting data and images generated from this pipeline

In [0]:
fiji = '/Applications/Fiji.app/Contents/MacOS/ImageJ-macosx'
codepath = '/Users/michaelshih/Documents/code/wucci/storm_image_processing'
datapath = '/Volumes/LaCie_DataStorage/xiaochao_wei_STORM imaging/STORM_imaging'
analysis_dir = 'analysis_20190419'


 ## Script name: `imgproc.py`
 * env: ImageJ/Fiji
 * rewritten: 04/26/2019
 * verified
 * output folder: `preprocessing`
 ### Output
   1. `preproimg` (`.tif`): seperate the channels into individual folders
   2. `imgintensity` (`.csv`): return mean intensity for each time frame from the center of the image (128 pixel * 128 pixel)
   3. `imginfo`:
       1. `imgmetadata` (`.txt`): return the metadata
       2. `imgstat.csv`: statistics from images
           * image_name: the name of the image without extension
           * ip_file_name: the name of the image with extension
           * xSize: frame size on the x-axis
           * ySize: frame size on the y-axis
           * nSlices: frame size on the z-axis
           * nFrames: number of time frames
           * nChannels: number of channles
           * size_MB: size of the file

In [0]:
script_name = 'imgproc.py'
ippath = '/Volumes/LaCie_DataStorage/xiaochao_wei_STORM imaging/STORM_imaging/resource/testdata'
opdir = 'preprocessing'
oppath = os.path.join(datapath, analysis_dir, opdir)

# create dict to store parameters
arg_dict = {
    'path': datapath,
    'dir_output': analysis_dir,
    'ippath': ippath,
    'outputdir': oppath,
    'batchmodeop': 'false',
}

arg = dict2arg_fiji(arg_dict)
print(arg)

In [0]:
# Run the script

In [0]:
subdir = 'Fiji'
scriptpath = os.path.join(codepath, subdir, script_name)
print("Start running: {}".format(script_name))
subprocess.check_output([fiji, '--ij2', '--run', arg])
print("End: {}".format(script_name))


 ## Script name: `tstormanalysis.py`
 * env: ImageJ/Fiji
 * plugin: [ThunderSTORM](https://github.com/zitmen/thunderstorm)
 * rewritten: 04/26/2019
 * verified
 * output folder: `tstorm`
 ### Output generated by ThunderSTORM
   1. `csvdata`:
       * (`.csv`): STORM dataset
       * (`.txt`): protocol
   2. `driftcorr` (`.json`): drift correction profile
   3. `histogram_plot` (`.tif`): histogram image with manification = 5.0

In [0]:
script_name = 'tstormanalysis.py'
ippath = '/Volumes/LaCie_DataStorage/xiaochao_wei_STORM imaging/STORM_imaging/analysis_20190419/preprocessing/preproimg'
opdir = 'preprocessing'
opsubdir = 'preproimg'

arg_dict = {
    'path': datapath,
    'dir_output': analysis_dir,
    'ippath': ippath,
    'batchmodeop': 'false', 
}
arg = dict2arg_fiji(arg_dict)
print(arg)


In [0]:
subdir = 'Fiji'
scriptpath = os.path.join(codepath, subdir, script_name)
print("Start running: {}".format(script_name))
subprocess.check_output([fiji, '--ij2', '--run', scriptpath, arg])
print("End: {}".format(script_name))


 ## Script name: `csv_slicer_T.py`
 * env: ImageJ/Fiji
 * rewritten: 04/26/2019
 * verified
 * output folder: `csvdata_sliced`
 ### Output generated by ThunderSTORM
   * `csvdata_sliced_T` (`.csv`): STORM data from T0 to T1

In [0]:
script_name = 'csv_slicer_T.py'
nchannel = 2
T0 = 5000
T1 = 10001
arg_list = [datapath, analysis_dir,'tstorm','csvdata', nchannel, T0, T1, 'csvdata_sliced_T']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('python '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `csv_slicer_ROI.py`
 * env: ImageJ/Fiji
 * rewritten: 04/29/2019
 * verified
 * output folder: `csvdata_sliced`
 ### Output generated by ThunderSTORM
   * `csvdata_sliced_T_ROI` (`.csv`): STORM data cropped by given region of interest.

In [0]:
# csv_slicer_crop.py
script_name = 'csv_slicer_ROI.py'
arg_list = [datapath, analysis_dir,'tstorm','csvdata_sliced_T', 2, 3, 'csvdata_sliced_T_ROI']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('python '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `spatialanaylsis.R`
 * env: ImageJ/Fiji
 * rewritten: 04/29/2019
 * verified
 * output folder: `spacial_test`
 ### Output generated by ThunderSTORM
   * `spacialdata` (`.csv`): Results of Ripley's K
       * K_r: Ripley's K function $$\hat{K}(r)= \hat{\lambda}^{-1}\sum_{i}\sum_{j\neq1}\frac{\omega(l_i, l_j)I(d_{ij<r})}{N}$$
       * L_r: L-function $$\hat{L}(r)=\sqrt{\frac{\hat{K}(r)}{\pi}}$$
       * H_r: H-function $$\hat{H}(r)=\hat{L}(r)-r$$
   * `spacialdata_bi` (`.rda`):

In [0]:
# spatialanaylsis.R
script_name = 'spatialanaylsis.R'
arg_list = [datapath, analysis_dir,'tstorm','csvdata_sliced_T_ROI', 2, 3, 'spacial_test', 'spacialdata', 'spacialdata_bi']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('Rscript --vanilla '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


In [0]:
# plot_spatialdata.py
script_name = 'plot_spatialdata.py'
arg_list = [datapath, analysis_dir,'spacial_test','spacialdata', 2, 'plot']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('python '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))

